In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import numpy as np

%pip install datawrapper

Note: you may need to restart the kernel to use updated packages.


In [3]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="PU52xuCQowgE5BFprvdnpFXzd5Ql9ISOSLBk58eNR2ykPztw0yC5fpqLTfsZdhyR"
)

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

### Staffing ratios data

In [5]:
nursing_ratios_raw = pd.read_csv(
    "https://69c46ad6-7f7a-4e40-9f99-cdf68e8848bf.filesusr.com/ugd/a53784_d422b73592324d48ac944b66ef52fd1c.csv?dn=Raw%20Data%203-2-22.csv"
)

In [6]:
nursing_ratios_raw.columns = [
    "submission_datetime",
    "title",
    "unit",
    "normal_ratio",
    "surged_ratio",
    "surge_frequency",
    "shift",
    "facility_name",
    "city",
    "state",
    "comments",
    "adequate_support_staff",
]

nursing_ratios_raw.head()

,submission_datetime,title,unit,normal_ratio,surged_ratio,surge_frequency,shift,facility_name,city,state,comments,adequate_support_staff
0,2022-02-24T06:40:29Z,Registered Nurse,Progressive Care,5:1,5:1,Multiple times a week,NaN,Banner Baywood Medical Center,Chandler,Arizona,Absolutely powerless. I feel as if I’m letting...,No
1,2022-02-23T22:21:04Z,Registered Nurse,Emergency Room,4:1,4:1,Multiple times a week,NaN,Candler Hospital,Savannah,GA,NaN,Sometimes
2,2022-02-24T05:17:05Z,Registered Nurse,Maternal Care,3:1,4:1,Once every few months,NaN,The Christ Hospital Liberty Birthing Center,Liberty Township,Ohio,NaN,Yes
3,2022-02-23T21:57:16Z,Registered Nurse,Intensive Care Unit,2:1,4:1,Once a week,NaN,DRK Berlin Koepenick,Berlin,Germany,NaN,No
4,2022-02-24T01:59:37Z,Registered Nurse,Med-Surg Tele,5:1,6:1,Multiple times a week,NaN,Grand View Hospital,Sellersville,PA,NaN,No


### State data

In [7]:
states = pd.read_csv(
    "https://raw.githubusercontent.com/stiles/notebooks/5a9a1a8395c7924abe6c7392fdabd58fda8d8d8d/census/raw/ap_states.csv"
)

In [8]:
states["name"] = states["name"].str.upper()

### Cleaning up data
##### how many total entries
##### how many diff entries for facilities

In [9]:
len(nursing_ratios_raw)

339

#### replicated matt's method for cleaning states but joining directly to the raw ratios file
#### let's see where we can get with fixing facility names

In [10]:
nursing_ratios_raw["state"] = nursing_ratios_raw["state"].str.upper()

In [11]:
nursing_ratios_raw.head()

,submission_datetime,title,unit,normal_ratio,surged_ratio,surge_frequency,shift,facility_name,city,state,comments,adequate_support_staff
0,2022-02-24T06:40:29Z,Registered Nurse,Progressive Care,5:1,5:1,Multiple times a week,NaN,Banner Baywood Medical Center,Chandler,ARIZONA,Absolutely powerless. I feel as if I’m letting...,No
1,2022-02-23T22:21:04Z,Registered Nurse,Emergency Room,4:1,4:1,Multiple times a week,NaN,Candler Hospital,Savannah,GA,NaN,Sometimes
2,2022-02-24T05:17:05Z,Registered Nurse,Maternal Care,3:1,4:1,Once every few months,NaN,The Christ Hospital Liberty Birthing Center,Liberty Township,OHIO,NaN,Yes
3,2022-02-23T21:57:16Z,Registered Nurse,Intensive Care Unit,2:1,4:1,Once a week,NaN,DRK Berlin Koepenick,Berlin,GERMANY,NaN,No
4,2022-02-24T01:59:37Z,Registered Nurse,Med-Surg Tele,5:1,6:1,Multiple times a week,NaN,Grand View Hospital,Sellersville,PA,NaN,No


In [12]:
nursing_ratios_raw[["city", "drop"]] = (
    nursing_ratios_raw["city"].str.upper().str.split(",", expand=True)
)

In [13]:
ratios_states_merge = pd.merge(
    nursing_ratios_raw, states, left_on="state", right_on="name", how="left"
)

In [14]:
ratios_states_merge["place"] = np.where(
    ratios_states_merge["usps"].isnull(),
    ratios_states_merge["state"],
    ratios_states_merge["usps"],
)

In [15]:
ratios_states_merge.head()

,submission_datetime,title,unit,normal_ratio,surged_ratio,surge_frequency,shift,facility_name,city,state,comments,adequate_support_staff,drop,name,usps,ap,place
0,2022-02-24T06:40:29Z,Registered Nurse,Progressive Care,5:1,5:1,Multiple times a week,NaN,Banner Baywood Medical Center,CHANDLER,ARIZONA,Absolutely powerless. I feel as if I’m letting...,No,None,ARIZONA,AZ,Ariz.,AZ
1,2022-02-23T22:21:04Z,Registered Nurse,Emergency Room,4:1,4:1,Multiple times a week,NaN,Candler Hospital,SAVANNAH,GA,NaN,Sometimes,None,NaN,NaN,NaN,GA
2,2022-02-24T05:17:05Z,Registered Nurse,Maternal Care,3:1,4:1,Once every few months,NaN,The Christ Hospital Liberty Birthing Center,LIBERTY TOWNSHIP,OHIO,NaN,Yes,None,OHIO,OH,Ohio,OH
3,2022-02-23T21:57:16Z,Registered Nurse,Intensive Care Unit,2:1,4:1,Once a week,NaN,DRK Berlin Koepenick,BERLIN,GERMANY,NaN,No,None,NaN,NaN,NaN,GERMANY
4,2022-02-24T01:59:37Z,Registered Nurse,Med-Surg Tele,5:1,6:1,Multiple times a week,NaN,Grand View Hospital,SELLERSVILLE,PA,NaN,No,None,NaN,NaN,NaN,PA


In [16]:
ratios_states_merge[ratios_states_merge["place"].isnull()]

,submission_datetime,title,unit,normal_ratio,surged_ratio,surge_frequency,shift,facility_name,city,state,comments,adequate_support_staff,drop,name,usps,ap,place


#### drop longer place names bc it means they are intl and drop columns you don't need

In [17]:
us_ratios = ratios_states_merge[ratios_states_merge["place"].str.len() == 2].drop(
    ["drop", "name", "usps", "ap"], axis=1
)

#### now do one where you get the state name

In [18]:
usratios_states_merge = pd.merge(
    us_ratios, states, left_on="place", right_on="usps", how="left"
)

In [19]:
usratios_states_merge[usratios_states_merge["name"].isnull()]

,submission_datetime,title,unit,normal_ratio,surged_ratio,surge_frequency,shift,facility_name,city,state,comments,adequate_support_staff,place,name,usps,ap


In [20]:
usratios_states_merge.head()

,submission_datetime,title,unit,normal_ratio,surged_ratio,surge_frequency,shift,facility_name,city,state,comments,adequate_support_staff,place,name,usps,ap
0,2022-02-24T06:40:29Z,Registered Nurse,Progressive Care,5:1,5:1,Multiple times a week,NaN,Banner Baywood Medical Center,CHANDLER,ARIZONA,Absolutely powerless. I feel as if I’m letting...,No,AZ,ARIZONA,AZ,Ariz.
1,2022-02-23T22:21:04Z,Registered Nurse,Emergency Room,4:1,4:1,Multiple times a week,NaN,Candler Hospital,SAVANNAH,GA,NaN,Sometimes,GA,GEORGIA,GA,Ga.
2,2022-02-24T05:17:05Z,Registered Nurse,Maternal Care,3:1,4:1,Once every few months,NaN,The Christ Hospital Liberty Birthing Center,LIBERTY TOWNSHIP,OHIO,NaN,Yes,OH,OHIO,OH,Ohio
3,2022-02-24T01:59:37Z,Registered Nurse,Med-Surg Tele,5:1,6:1,Multiple times a week,NaN,Grand View Hospital,SELLERSVILLE,PA,NaN,No,PA,PENNSYLVANIA,PA,Pa.
4,2022-02-24T09:50:17Z,Registered Nurse,Med-Surg Tele,4:1,5:1,Once every few weeks,NaN,Methodist Hospital,SAN ANTONIO,TEXAS,NaN,Sometimes,TX,TEXAS,TX,Texas


#### now you don't need state bc you have state name correct from states, so drop unnecessary ones and then rename ones you want

In [21]:
usratios_cleaned = usratios_states_merge.drop(["state", "usps", "ap"], axis=1).rename(
    columns={"name": "state", "place": "state_abbv"}
)

#### let's see where we can get with fixing facility names

In [22]:
# usratios_cleaned.sort_values("facility_name")

#### not totally sure i can publish names of hospitals with this ratios without varifying--maybe better as a table showing the range
#### or by state showing which report the worst (do california's mandated ratios work??)
#### will return to cleaning facility names in order to identify the number of 

In [23]:
usratios_cleaned[usratios_cleaned["unit"] == "SNF"].value_counts(
    "normal_ratio"
).reset_index()

,normal_ratio,0
0,20+:1,5
1,10:1,2
2,14:1,2
3,18:1,1
4,30+:1,1
5,8:1,1


In [24]:
usratios_cleaned[usratios_cleaned["unit"] == "SNF"].value_counts(
    "surged_ratio"
).reset_index()

,surged_ratio,0
0,20+:1,8
1,12:1,1
2,15:1,1
3,19:1,1
4,30+:1,1


In [25]:
usratios_cleaned.value_counts("surge_frequency").reset_index()

,surge_frequency,0
0,Multiple times a week,114
1,Once every few weeks,60
2,Once every few months,41
3,Once a week,35
4,Never,34
5,Every day,28
6,Once a month,17


In [26]:
ratio_states = usratios_cleaned.value_counts("state")

In [27]:
test = usratios_cleaned

#### translating ratios to values

In [28]:
test[["normal_ratio_val", "drop"]] = test["normal_ratio"].str.split(":", expand=True)
test[["surge_ratio_val", "surgedrop"]] = test["surged_ratio"].str.split(
    ":", expand=True
)

In [29]:
test["normal_ratio_clean"] = test["normal_ratio_val"].str.replace("+", "")
test["surge_ratio_clean"] = test["surge_ratio_val"].str.replace("+", "")

/var/folders/gg/6wnjn6cd1qlfst8mkw4g4xqc0000gn/T/ipykernel_16215/1367717645.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  test["normal_ratio_clean"] = test["normal_ratio_val"].str.replace("+", "")
/var/folders/gg/6wnjn6cd1qlfst8mkw4g4xqc0000gn/T/ipykernel_16215/1367717645.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  test["surge_ratio_clean"] = test["surge_ratio_val"].str.replace("+", "")


In [30]:
test.head()

,submission_datetime,title,unit,normal_ratio,surged_ratio,surge_frequency,shift,facility_name,city,comments,adequate_support_staff,state_abbv,state,normal_ratio_val,drop,surge_ratio_val,surgedrop,normal_ratio_clean,surge_ratio_clean
0,2022-02-24T06:40:29Z,Registered Nurse,Progressive Care,5:1,5:1,Multiple times a week,NaN,Banner Baywood Medical Center,CHANDLER,Absolutely powerless. I feel as if I’m letting...,No,AZ,ARIZONA,5,1,5,1,5,5
1,2022-02-23T22:21:04Z,Registered Nurse,Emergency Room,4:1,4:1,Multiple times a week,NaN,Candler Hospital,SAVANNAH,NaN,Sometimes,GA,GEORGIA,4,1,4,1,4,4
2,2022-02-24T05:17:05Z,Registered Nurse,Maternal Care,3:1,4:1,Once every few months,NaN,The Christ Hospital Liberty Birthing Center,LIBERTY TOWNSHIP,NaN,Yes,OH,OHIO,3,1,4,1,3,4
3,2022-02-24T01:59:37Z,Registered Nurse,Med-Surg Tele,5:1,6:1,Multiple times a week,NaN,Grand View Hospital,SELLERSVILLE,NaN,No,PA,PENNSYLVANIA,5,1,6,1,5,6
4,2022-02-24T09:50:17Z,Registered Nurse,Med-Surg Tele,4:1,5:1,Once every few weeks,NaN,Methodist Hospital,SAN ANTONIO,NaN,Sometimes,TX,TEXAS,4,1,5,1,4,5


In [69]:
ratio_values = test[
    [
        "submission_datetime",
        "facility_name",
        "city",
        "state",
        "state_abbv",
        "title",
        "unit",
        "normal_ratio_clean",
        "surge_ratio_clean",
        "surge_frequency",
        "shift",
        "adequate_support_staff",
        "comments",
    ]
]

ratio_values = ratio_values[ratio_values["title"] == "Registered Nurse"]

In [70]:
ratio_values[["normal_ratio_clean", "surge_ratio_clean"]] = ratio_values[
    ["normal_ratio_clean", "surge_ratio_clean"]
].astype(int)

In [87]:
er_ratio = ratio_values[ratio_values["unit"] == "Emergency Room"]
icu_ratios = ratio_values[ratio_values["unit"] == "Intensive Care Unit"]
snf_ratios = ratio_values[ratio_values["unit"] == "SNF"]

In [77]:
state_ratios = (
    ratio_values.groupby(["state", "state_abbv"])
    .agg(
        mean_normal_ratio=("normal_ratio_clean", "mean"),
        median_normal_ratio=("normal_ratio_clean", "median"),
        mean_surge_ratio=("surge_ratio_clean", "mean"),
        median_surge_ratio=("surge_ratio_clean", "median"),
        count=("submission_datetime", "count"),
    )
    .reset_index()
)

In [82]:
city_ratios = (
    ratio_values.groupby(["city", "state", "state_abbv"])
    .agg(
        mean_normal_ratio=("normal_ratio_clean", "mean"),
        median_normal_ratio=("normal_ratio_clean", "median"),
        mean_surge_ratio=("surge_ratio_clean", "mean"),
        median_surge_ratio=("surge_ratio_clean", "median"),
        count=("submission_datetime", "count"),
    )
    .reset_index()
)

In [81]:
dw.add_data(chart_id="LV85q", data=state_ratios)

<Response [204]>

In [85]:
city_ratios[city_ratios["count"] > 1].sort_values(
    "median_normal_ratio", ascending=False
)

In [101]:
er_ratio_counts = (
    er_ratio.groupby("normal_ratio_clean")
    .agg(normal_count=("normal_ratio_clean", "count"))
    .reset_index()
)

In [102]:
dw.add_data(chart_id="baeE6", data=er_ratio_counts)

<Response [204]>

In [103]:
er_surge_ratio_counts = (
    er_ratio.groupby("surge_ratio_clean")
    .agg(surge_count=("surge_ratio_clean", "count"))
    .reset_index()
)